# Desafio:  California Housing Prices

## Objetivos:

O objetivo desse desafio é analisar os dados do case e estruturar uma Feature Engineering básica apenas com os dados existentes, sem transformar ou combinar features ou mesmo adicionar informações externas. 

Ao final do desafio, será treinado um modelo de regressão linear com as features obtidas. Esse modelo será testado contra uma massa de teste, separada previamente.


## Sobre o Case

### Case baseado no dataset do Kaggle: "California Housing Prices"

Esse desafio é baseado em um dataset aberto do Kaggle ([https://www.kaggle.com](https://www.kaggle.com)) de 2018, de onde é possível estimar o preço de um imóvel pertencente a uma dada região na Califórnia. 

O dataset original foi extraído do repositório StatLib, que não está mais disponível. Os dados que o compôem foram retirados do Censo realizado na Califórnia em 1990 e modificado para servir como base de treinamento.


Link para o dataset no Kaggle: [https://www.kaggle.com/harrywang/housing/data](https://www.kaggle.com/harrywang/housing/data)


### Descrição dos Dados Originais:

#### Tamanho do Dataset:

* `20.640` data points

#### Variável dependente:

* `median_house_value`:  (float) variável dependente com o valor da mediana do preço de imóvel na região

#### Features: 

* `longitude`/`latitude`: (floats) posição global da região
* `housing_median_age`: (float) mediana da idade (em anos) das casas da região
* `total_rooms`: (float) total de aposentos da região
* `total_bedrooms`: (float) total de quartos da região
* `population`: (float) população total da região
* `households`: (float) quantidade total de imóveis da região
* `median_income`: (float) mediana do salário (por hora) de uma pessoa na região
* `ocean_proximity`: (string) categorias relativas à distância do oceano

É interessante ressaltar que `207` valores foram retirados aleatoriamnete da feature `total_bedrooms` por motivos acadêmicos.


### Modificação dos dados para o Desafio:

Para tornar o desafio mais fácil de avaliar, a massa de dados original foi dividida em duas massas, uma para treino e outra para teste, ambas contendo `10.320` elementos.

Também foram removidos aleatoriamente ainda mais valores da massa de dados e em mais features. Isso torna o processo de Feature Engineering mais instrutivo, pois aproxima-se da realidade do trabalho de um Data Scientist.

___

# Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, PolynomialFeatures, StandardScaler

# Dataset:

## Carregando os dados

In [ ]:
dataset = pd.read_csv("data/california_housing_train.csv", index_col=0)
x_train = dataset.drop(["median_house_value"], axis=1)
y_train = dataset[["median_house_value"]]

In [ ]:
print(f"shape: {x_train.shape}")
x_train.head()

In [ ]:
print(f"shape: {y_train.shape}")
y_train.head()

In [ ]:
dataset = pd.read_csv("data/california_housing_test.csv", index_col=0)
x_test = dataset.drop(["median_house_value"], axis=1)
y_test = dataset[["median_house_value"]]

In [ ]:
print(f"shape: {x_test.shape}")
x_test.head()

In [ ]:
print(f"shape: {y_test.shape}")
y_test.head()

# Problemas

## A) Features Numéricas

Features numéricas são as primeiras a serem tratadas, por serem as mais fáceis de compreender e de relacionar com o problema. As seções a seguir focam no tratamento e limpeza dessas features, o primeiro passo na engenharia de features.


### Distribuição das Features

In [ ]:
""" Complete os espaços com ? """
numerical_cols = [?, ?, ..., ?]

In [ ]:
# descrevendo as distribuições
cuts = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
x_train[numerical_cols].describe(percentiles=cuts)

Para tomada de decisão, é sempre mais informativo usar visualizações. Com elas é possível observar o formato da distribuição (normal, exponencial, logarítmica, etc) e os outliers.

Verificando as distribuições com histogramas:

In [ ]:
x_train[numerical_cols].hist(bins=50, figsize=(15,10))

###  Detecção e Tratamento de Nulos

Conhecendo a distribuição das features, já é possível traçar estratégias de tratamento de valores nulos. A primeira tarefa é detectar a proporção de nulos nos dados.


* Verificar quantidade de Nulos em cada feature
* Entender que Nulos podem aparecer também nos dados de teste, portanto apenas descartar o dado não resolve
* Entender que medida (média ou mediana) deve ser imputada em cada feature

In [ ]:
x_null = x_train[numerical_cols].isnull()
null_data = pd.DataFrame({
    "count": x_null.sum(),
    "mean": x_null.mean()
})
null_data

Normalmente a ocorrência de valores nulos ocorre por algum ruído no processo de obtenção de dados, o que significa que muito provavelmente eles ocorrerão também em dados de produção.

Existem algumas técnicas de tratamento de valores nulos:

- Atribuir um valor pafrão fora da distribuição;
- Criar modelos para inferir os valores a partir das outras features;
- Imputar um valor referente à distribuição:
    - média
    - mediana
    
Para features numéricas em que conhecemos a distribuição mas não foram tratados os outliers, o mais recomendado é utilizar a **mediana**, cujo valor é pouco afetado por outliers.

In [ ]:
class NumericalFeaturesImputer(BaseEstimator, TransformerMixin):
    """ Classe de Feature Transformer baseada em um Imputer de Mediana.
        Esse imputer mantém a entrada X como um DataFrame em vez de transformar em numpy.array.
    """
    
    def __init__(self, columns):
        self.imputer = Imputer(strategy="median")
        self.columns = columns
        
    def fit(self, X, y=None, **fit_params):
        self.imputer.fit(X.loc[:, self.columns])
        return self
    
    def transform(self, X):
        X_t = X.copy()
        X_t.loc[:, self.columns] = self.imputer.transform(X_t.loc[:, self.columns])
        return X_t
    

In [ ]:
# sanity check 1: tranformation keeps columns 
numerical_imputer = NumericalFeaturesImputer(numerical_cols)
x_train = numerical_imputer.fit_transform(x_train)
x_train.head()

In [ ]:
# sanity check 2: NUll data replaced
x_null = x_train[numerical_cols].isnull()
null_data = pd.DataFrame({
    "count": x_null.sum(),
    "mean": x_null.mean()
})
null_data

### Transformação Logarítmica de Features

Com os histogramas, já fica evidente que a distribuição de algumas features é exponencial. A análise de outliers desse tipo de distribuição pode ser prejudicada pela alta concentração de elementos em uma pequena parte do domínio. 

Uma forma de corrigir essa distorção é transformar esses dados com a função **logarítmica**; a transformação com essa função torna a distribuição das features mais próxima da normal.

In [ ]:
""" Complete os espaços com ? """
log_cols = [?, ?, ..., ?]

In [ ]:
class LogFeaturesTransform(BaseEstimator, TransformerMixin):
    """ Classe de Feature Transformer para aplicar log em valores numéricos.
        Esse imputer mantém a entrada X como um DataFrame em vez de transformar em numpy.array.
    """
    
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X):
        X_t = X[self.columns].apply(np.log).rename(columns=lambda c: f"log_of_{c}")
        return X.join(X_t)
    

Ainda é cedo para descartar as features originais tratadas, pois elas podem ter ainda algum poder preditivo que pode ficar ocluso pela transformação. O descarte de features normalmente é feito quando se detecta multi-colinearidade entre as features ou durante uma etapa de _feature selection_.

Por hora, as novas features serão integradas ao dataset original.

In [ ]:
x_train = LogFeaturesTransform(log_cols).fit_transform(x_train)

In [ ]:
log_cols = [f"log_of_{c}" for c in log_cols]

In [ ]:
x_train[log_cols].hist(bins=50, figsize=(15,10))

###  Detecção e Remoção de Outliers

Outliers podem deformar a percepção do domínio para o aprendizado de um modelo linear, impedindo o mesmo de encontrar uma solução correta. 

#### Verificando os Box Plots para observar os Outliers

In [ ]:
""" Escreva a solução aqui """

#### Aplicando os cortes:

In [ ]:
# inicialização
keep_index = pd.Series(index=x_train.index, data=True)
cuts_table = pd.DataFrame(columns=["count", "percent"])

In [ ]:
# Template para os cortes: replique essa célula para cada corte definido no item anterior

""" Complete os espaços com ? """

feat = "?"
lim_inf = ?
lim_sup = ?

cuts_index = (x_train[feat] < lim_inf) | ((x_train[feat] > lim_sup)) 
cuts_table = cuts_table.append(
    pd.DataFrame(
        index=[f"{lim_inf} <= {feat} <= {lim_sup}"],
        columns=["count", "percent"],
        data=[[cuts_index.sum(), cuts_index.mean()]]
    )
)
keep_index &= ~cuts_index

In [ ]:
# visualizando os cortes
cuts_table.append(
    pd.DataFrame(
        index=[f"Total Elements Cut"],
        columns=["count", "percent"],
        data=[[(~keep_index).sum(), (~keep_index).mean()]]
    )
)

In [ ]:
# aplicação do corte total
ori_size = x_train.shape[0]

x_train = x_train.loc[keep_index]

new_size = x_train.shape[0]

print(f"Size of 'x_train' before Cuts:\t {ori_size}")
print(f"Size of 'x_train' after Cuts:\t {new_size} (-{100. * (~keep_index).mean(): 0.2f} %)")

## B) Feature Categórica

Features Categóricas são um pouco mais interessantes de tratar do que as numéricas, já que existem muitas maneiras de se transformar textos ou símbolos em valores numéricos. 

Vale lembrar que todo modelo de machine learning compreende o mundo através de valores numéricos, por serem modelos matemáticos de busca de solução ótima. Alguns frameworks atuais permitam que se coloquem valores simbólicos ou de texto marcados com a tag 'category' diretamente no dataset, mas por trás o proprio framework transforma esses dados em números.

### Análise da Distribuição das Categorias

#### Verificação da quantidade de dados em cada categoria

É interessante verificar a quantidade de dados em cada categoria, pois categoriass mal representadas podem criar conceitos enviesados do modelo sobre o domínio. Por exemplo, em um dataset em que uma categoria só ocorra uma única vez e a variável dependente exatamente nesse elemento seja muito alta, um modelo treinado pode assumir que a presença dessa categoria já indique uma saída alta.

In [ ]:
# verificando a distribuiçlão das categorias na massa de treino
x_train["ocean_proximity"].fillna(" - NaN - ").value_counts()

Nem sempre é possível verificar a distribuição exata dos dados de produção, mas a massa de teste normalmente dá uma boa aproximação dela. 

In [ ]:
# verificando a distribuiçlão das categorias na massa de teste
x_test["ocean_proximity"].fillna(" - NaN - ").value_counts().index

#### Definir o tratamento de categorias pouco representativos:

Pode-se observar que a categoria `ISLAND` tem uma representatividade mínima em todo o dataset, tornando essa categoria a única candidata à eliminação. Como já existem elementos com a categoria nula nesse dataset, a melhor estratégia é juntar a categoria `ISLAND` aos nulos e tratá-los (próxima seção).

###  Detecção e Tratamento de Nulos

Como já foram identificados elementos em que a categoria é nula, é importante tratar esses elementos apropriadamente.

Existem algumas estratégias para tratamento de nulos:

- Criar uma categoria `NULL` e usar como um símbolo válido do sistema;
- Criar modelos para inferir os valores a partir das outras features;
- Imputar um valor referente à distribuição: a `moda` (valor com a maior frequência)

Como existe a informação de que a variável categórica foi criada a partir da anotação manual do autor do dataset e que o mesmo utilizou as coordenadas `latitude` e `longitude`, a melhor estratégia é criar um `Imputer` que **busque a categoria do elemento usando as coordenadas geográfica**. 

In [ ]:
""" Complete os espaços com ? """
from sklearn.linear_model import LogisticRegression


class CategoricalFeaturesImputer(BaseEstimator, TransformerMixin):
    """ Classe de Feature Transformer baseada em um Imputer Categórico que busca a categoria 
        do elemento mais próximo.
        Esse imputer mantém a entrada X como um DataFrame em vez de transformar em numpy.array.
    """
    
    def __init__(self, valid_categories):
        self.valid_categories = valid_categories
        self.imputer = ?
        
    def fit(self, X, y=None, **fit_params):
        target_feat = ?
        inputs_cols = [?, ?]
        
        index = X[target_feat].isin(self.valid_categories)
        
        x_train = X.loc[index, inputs_cols]
        y_train = X.loc[index, target_feat]
        
        self.imputer.fit(x_train, y_train)
        return self
    
    def transform(self, X):
        target_feat = ?
        inputs_cols = [?, ?]
        
        X_t = X.copy()
        X_t.loc[:, target_feat] = self.imputer.predict(X_t.loc[:, inputs_cols])
        return X_t
    

In [ ]:
# sanity check

""" Complete os espaços com ? """

valid_categories = [?, ?, ..., ?]

x_valid = x_test[x_test.ocean_proximity.isnull()]

obj = CategoricalFeaturesImputer(valid_categories).fit(x_train)
obj.transform(x_valid)

###  Transformação em Dummy Features

A transformação em Dummy Features é a técnica em que os labels são formatados em dados categóricos consumíveis pelo modelo. O formato mais comum é usar uma representação em que cada label é uma nova feature binária onde o valor é **um** onde a feature é igual ao label e **zero** em todo o resto. 

Por exemplo, a transformação do vetor `[a, b, d, b, e, a, c]` seria da forma:

| a | b | c | d | e |
|---|---|---|---|---|
| 1 | 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 0 | 0 | 1 | 0 |
| 0 | 1 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 1 |
| 1 | 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 | 0 |

Em modelos lineares existe uma regra de ouro que uma das classes deve permanecer como 'Base' para não haver uma feature linearmente dependente dentro do dataset. Alguns modelos tratam esse problema internamente, mas ainda assim é uma boa prática a ser seguida.

Como é uma etapa de pré-processamento, essa transformação também deve ser feita como uma Feature Transformer.

In [ ]:
class CategoricalToDummyFeaturesTransform(BaseEstimator, TransformerMixin):
    """ Classe de Feature Transformer que transforma labels na representação Dummy.
        Esse imputer mantém a entrada X como um DataFrame em vez de transformar em numpy.array.
    """
    
    def __init__(self, categories):
        self.categories = [f"ocean_proximity: {c}" for c in categories]
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X):        
        X_t = X.drop("ocean_proximity", axis=1)
        
        dummy = pd.get_dummies(
            X["ocean_proximity"], 
            prefix="ocean_proximity",
            prefix_sep=": "
        ) 
        return X_t.join(dummy.loc[:, self.categories])
    

In [ ]:
# sanity check

""" Complete os espaços com ? """

categories = [?, ?, ..., ?]

CategoricalToDummyFeaturesTransform(categories).transform(x_train).head(10)

## C) Combinação Manual de Features

É uma prática comum agrupar manualmente features por assunto ou grupos de conhecimento, principalmente quando se tem um bom conhecimento do domínio de aplicação. 

Algumas features numéricas nesse dataset são bem relacionadas entre si, sendo imediato pensar em combiná-las. Para manter o padrão no pré-processamento, isso será feito em uma classe de Feature Transformer.

In [ ]:
""" Complete os espaços com ? """

class ManuallyCraftedFeaturesTransform(BaseEstimator, TransformerMixin):
    """ Classe de Feature Transformer que aplica transformações manuais.
        Esse imputer mantém a entrada X como um DataFrame em vez de transformar em numpy.array.
    """    
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X):        
        X_t = X.copy()
        
        # Início das transformações manuais; 
        # repita as linhas abaixo para cada transformação
        
        X_t.loc[:, "?"] = ? * ? / ? + ? 
        
        # Fim das transformações
        
        return X_t
    

In [ ]:
# sanity check

ManuallyCraftedFeaturesTransform().transform(x_train).head(10)

## D) Combinação Polinomial de Features

Uma outra técnica muito utilizada em feature engineering é criar novas features a partir da combinação polinomial de features antigas. Dessa forma, uma solução linear pode ser extendida para uma solução não linear.

In [ ]:
class PolynomialFeaturesTransform(BaseEstimator, TransformerMixin):
    """ Classe de Feature Transformer que cria features não lineares usando Transformação Polinomial.
        Esse imputer mantém a entrada X como um DataFrame em vez de transformar em numpy.array.
    """
    
    def __init__(self, features, degree):
        self.features = features
        self.polifeat = PolynomialFeatures(degree=degree, include_bias=False)
        
    def fit(self, X, y=None, **fit_params):
        self.polifeat.fit(X[self.features])
        return self
    
    def transform(self, X):        
        X_t = X.drop(self.features, axis=1)
        
        x_poli = pd.DataFrame(
            index=X.index,
            columns=self.polifeat.get_feature_names(self.features),
            data=self.polifeat.transform(X[self.features])
            
        )
        
        return X_t.join(x_poli)
    

In [ ]:
# sanity check

""" Complete os espaços com ? """

features = [?, ?, ..., ?]

PolynomialFeaturesTransform(features, 4).fit_transform(x_train).head(10)

## E) Dados Externos: Pontos de Interesse

Uma das técnicas que mais trazem informação para o modelo é a inclusão de dados externos. 

Para essa seção, deve-se escolher alguns pontos de interesse da Califórina e arredores para calcular a distância euclidiana da Latitude/Longitude do data point a esses pontos de interesse.

In [ ]:
""" Complete os espaços com ? """

class PointsOfInterestFeaturesTransform(BaseEstimator, TransformerMixin):
    """ Classe de Feature Transformer que cria features baseadas na distância a pontos de interesse.
        Esse imputer mantém a entrada X como um DataFrame em vez de transformar em numpy.array.
    """
    
    def __init__(self):
        self.poi = {
            "?": (?, ?),
            "?": (?, ?),
            "?": (?, ?)
        }
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X):        
        X_t = X.copy()        
        x_poi = pd.DataFrame(index=X_t.index)
        for k in self.poi:
            distance_lat = (X_t.latitude - self.poi[k][0]) ** 2
            distance_lon = (X_t.longitude - self.poi[k][1]) ** 2
            x_poi.loc[:, f"Distance to {k}"] = (distance_lat + distance_lon) ** 0.5        
        return X_t.join(x_poi)
    

In [ ]:
PointsOfInterestFeaturesTransform().fit_transform(x_test.head())

## F) Treinamento e Avaliação de um Modelo Linear

O modelo de machine learning é apenas a parte final de um pipeline de processamentos; o pipeline completo é formado por todas as etapas de pré-processamento desde o dado bruto até as etapas de normalização e redução de dimensionalidade, finalizado pelo modelo preditivo.

O framework `Scikit-Learn` implementa uma ferramenta que permite a montagem de um pipeline completo, que pode ser treinado e usado para predição como um objeto único, que pode ser inclusive salvo em um arquivo. Isso permite que todo o pipeline possa ser exportado para produção sem ser reimplementado.

###  Reload das massas de Treino e de Teste

As massas de dados de Treino e de Teste serão carregadas novamente para que seja aplicado o pipeline de pré-processamento em ambos desde o princípio. 

In [ ]:
dataset = pd.read_csv("data/california_housing_train.csv", index_col=0)
x_train = dataset.drop(["median_house_value"], axis=1)
y_train = dataset[["median_house_value"]]

In [ ]:
dataset = pd.read_csv("data/california_housing_test.csv", index_col=0)
x_test = dataset.drop(["median_house_value"], axis=1)
y_test = dataset[["median_house_value"]]

###  Limpeza do dataset de treino

Devem ser aplicados os mesmos cortes definidos no item A, para que apenas os dados dentro da distribuição correta sejam usados para o treinamento.

In [ ]:
cuts_table

In [ ]:
x_train = x_train.loc[keep_index]

In [ ]:
y_train = y_train.loc[keep_index]

###  Pipeline de Pré-Processamento

A construção do pipeline deve incluir:

* Todas as etapas de pré-processamento
* Normalização (Z-Score)
* Modelo Linear

In [ ]:
""" Complete os espaços com ? """
numerical_features = [?, ?, ..., ?]
log_transform_features = [?, ?, ..., ?]
imputer_categories = [?, ?, ..., ?]
dummy_categories = [[?, ?, ..., ?]
poly_features = [?, ?, ..., ?]
poly_degree = ?

In [ ]:
pipeline = Pipeline([
    ("numerical_imputer",          NumericalFeaturesImputer(numerical_features)),
    ("logarithmic_transform",      LogFeaturesTransform(log_transform_features)),
    ("categorical_imputer",        CategoricalFeaturesImputer(imputer_categories)),
    ("dummy_category_transform",   CategoricalToDummyFeaturesTransform(dummy_categories)),
    ("manually_crafted_transform", ManuallyCraftedFeaturesTransform()),
    ("polynomial_feats_transform", PolynomialFeaturesTransform(poly_features, poly_degree)),
    ("points_of_interest",         PointsOfInterestFeaturesTransform()),
    ("zscore",                     StandardScaler()),
    ("predictor",                  ElasticNet()),
])

###  Treinar e avaliar o modelo

In [ ]:
# trainamento
pipeline.fit(x_train, y_train)

In [ ]:
# avaliação do modelo na massa de treino
y_true = y_train
y_pred = pipeline.predict(x_train)
mse_tr = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_tr = r2_score(y_true=y_true, y_pred=y_pred)
r2_tr = 1 - (1-r2_tr)*(len(y_train)-1)/(len(y_train)-x_train.shape[1]-1)

In [ ]:
# avaliação do modelo na massa de teste
y_true = y_test
y_pred = pipeline.predict(x_test)
mse_te = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_te = r2_score(y_true=y_true, y_pred=y_pred)
r2_te = 1 - (1-r2_te)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)

In [ ]:
# tabela final mostrando os resultados
pd.DataFrame(
    index=["train", "test"],
    columns=["MSE", "R^2"],
    data=[
        [mse_tr, r2_tr],
        [mse_te, r2_te]
    ]
)

In [ ]:
# tabela final mostrando os resultados
pd.DataFrame(
    index=["train", "test"],
    columns=["MSE", "R^2"],
    data=[
        [mse_tr, r2_tr],
        [mse_te, r2_te]
    ]
)